# Создание датасета паронимов

Будем строить датасет на основе русского [онлайн-словаря паронимов](https://ru-paronym.ru). 

На выходе мы хотим получить файл, выглядищий следующим образом:



```
{
  'paronym1': {'paronym2': <link>},
  'paronym2': {'paronym1': <link>},
  'paronym3': {'paronym4': <link>},
  ...
}
```
Т.е. для каждого паронима в соответствие ставится его пароним и ссылка на их общую страницу.


In [70]:
import requests
import json
from bs4 import BeautifulSoup


Для того, чтобы обойти все буквенные страницы в словаре, делаем массив русского алфавита. Заметим, что не для всех букв страницы в словаре есть, так что во избежание лишних ошибок будем пропускать их.

In [65]:
paronyms = {}
home_url = 'https://ru-paronym.ru'
ru_alphabet = [chr(i) for i in range (ord('А'), ord('А') + 32)]
ru_alphabet

['А',
 'Б',
 'В',
 'Г',
 'Д',
 'Е',
 'Ж',
 'З',
 'И',
 'Й',
 'К',
 'Л',
 'М',
 'Н',
 'О',
 'П',
 'Р',
 'С',
 'Т',
 'У',
 'Ф',
 'Х',
 'Ц',
 'Ч',
 'Ш',
 'Щ',
 'Ъ',
 'Ы',
 'Ь',
 'Э',
 'Ю',
 'Я']

In [58]:
no_words = ['Ё', 'Й', 'Щ', 'Ъ', 'Ы', 'Ь']

In [66]:
for letter in ru_alphabet:
    if letter in no_words:
        continue
    url = home_url + '/' + letter
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    arr = soup.find_all('a', href=True)[2:-27]  # Откидываем первые 2 и последние 27 элемента, которые есть на каждой странице
    for elem in arr:
        webpage = home_url + elem['href']
        words = (elem.string).split(' — ')
        if words[0] not in paronyms.keys():
            paronyms[words[0]] = {}
        if words[1] not in paronyms.keys():
            paronyms[words[1]] = {}
        paronyms[words[0]][words[1]] = webpage
        paronyms[words[1]][words[0]] = webpage

In [69]:
paronyms

{'абонемент': {'абонент': 'https://ru-paronym.ru/абонемент-абонент'},
 'абонент': {'абонемент': 'https://ru-paronym.ru/абонемент-абонент'},
 'абстрактность': {'абстракция': 'https://ru-paronym.ru/абстрактность-абстракция'},
 'абстракция': {'абстрактность': 'https://ru-paronym.ru/абстрактность-абстракция'},
 'авантюрин': {'авантюрист': 'https://ru-paronym.ru/авантюрин-авантюрист'},
 'авантюрист': {'авантюрин': 'https://ru-paronym.ru/авантюрин-авантюрист'},
 'автобиографический': {'автобиографичный': 'https://ru-paronym.ru/автобиографический-автобиографичный'},
 'автобиографичный': {'автобиографический': 'https://ru-paronym.ru/автобиографический-автобиографичный'},
 'автоматический': {'автоматичный': 'https://ru-paronym.ru/автоматический-автоматичный'},
 'автоматичный': {'автоматический': 'https://ru-paronym.ru/автоматический-автоматичный'},
 'автономия': {'автономность': 'https://ru-paronym.ru/автономия-автономность'},
 'автономность': {'автономия': 'https://ru-paronym.ru/автономия-авто

In [71]:
with open("paronyms.json", "w") as f:
    json.dump(paronyms, f)

# Выделение паронимов в предложениях

Используя уже имеющийся датасет предложений, выделим те, в которых используются паронимы.

In [96]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 50.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=2eb2cb2fd7e7725828de09b014f44e9b73b9a1f7438a9a55a31a25f1c63e8a4e
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [128]:
import jsonlines
import pandas
import pymorphy2
from tqdm import tqdm

In [98]:
morph = pymorphy2.MorphAnalyzer()

In [76]:
with jsonlines.open('alisa_selezneva.jsonl') as f:
    sentences = pandas.DataFrame(f)

In [127]:
def morph_list(words, n):
    final = []
    for w in words:
        p = morph.parse(w)[0]
        if 'NOUN' in parse_word.tag:
            final.append(p.inflect({cases[n]}).word)
    return final

In [132]:
f = jsonlines.open('paronyms_alisa.jsonl', mode='w')
cases = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']
all = sentences['sentence'].to_list()[:-1]
parlist = list(paronyms.keys())
new_sentences = {}
for j in tqdm(range(len(all))):
    now_sent = {'sentence': all[j], 'paronyms': []}
    for word in parlist:
        # parse_word = morph.parse(word)[0]
        # word_cases = []
        # if 'NOUN' in parse_word.tag:
        #     for i in cases:
        #         word_cases.append(parse_word.inflect({i}).word)
        # else:
        #     word_cases = [word]
        # for i in range(len(word_cases)):
        #     if word_cases[i] in all[j].split():
        #         if 'NOUN' in parse_word.tag:
        #             pars = morph_list(list(paronyms[word].keys()), i)
        #         else:
        #             pars = list(paronyms[word].keys())
        #         now_sent['paronyms'].append({'word': word_cases[i], 'paronyms': pars})
        if word in all[j].split():
            now_sent['paronyms'].append({'word': word, 'paronyms': list(paronyms[word].keys())})
    if len(now_sent['paronyms']) > 0:
        f.write(now_sent)
f.close()

100%|██████████| 10000/10000 [00:26<00:00, 383.13it/s]
